In [3]:
# Load a variety of modules
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy # Must be installed using Anaconda Prompt
import json
import pandas as pd
import csv
import re
#from textblob import TextBlob
import string
#import preprocessor as p
import os
import time

In [4]:
#Apply for access to a twitter developer account here: https://developer.twitter.com/en/apply-for-access

# These are the keys you get with a twitter developer account
consumer_key = "xxxxxxxxxxxxxxxxxxxx"
consumer_secret = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
access_token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
access_token_secret = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# Set your authorizaton access in tweepy from your twitter developer account
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [7]:
#https://medium.com/python-in-plain-english/scraping-tweets-with-tweepy-python-59413046e788

def scrapetweets(search_words, date_since, numTweets, numRuns):
    # Define a for loop to generate tweets at regular intervales
    # We cannot make large API calls in one go, so we try T times
    #Define a pandas df to store the data
    db_tweets = pd.DataFrame(columns = ['username','acctdesc','location','following','followers',
                                       'totaltweets','usercreatedts','tweetcreatedts','retweetcount',
                                       'text','hashtags'])
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run
        start_run = time.time()
        
        # Collect tweets using the Cursor Object
        # .Cursor() returns and object that you can iterate or loop over to acces the data collected.
        # Each item in the ieterator has various attributes that you can access for each tweet.
        tweets = tweepy.Cursor(api.search, q = search_words, lang = "en", since = date_since,
                              tweet_mode = 'extended').items(numTweets)
        
        #Store these tweets in a python list
        tweet_list = [tweet for tweet in tweets]
        
        # Obtain the following information
            # user.screen_name = twitter handle
            # user.description = description of the account
            # user.location = where he/she is tweeting from
            # user.friends_count = number of users this user is following
            # user.follower_count = number of users that are following this user
            # user.statusts_count = total tweets by user
            # user.created_at = When the user account was created
            # created_at = when the tweet was created
            # retweet_count = number of retweets
            # retweeted_status.full_text = full text of the tweet
            # tweet.entitiies['hashtags'] = hashtags in the tweet
            # tweet.entities['media'] = any media linked in the tweet (photos, videos, gifs, etc)
            
        # Begin scraping individual tweets
        noTweets = 0
        
        for tweet in tweet_list:
            # Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']

            try:
                text = tweet.retweeted_status.full_text
            except AttributeError: #Not a retweet
                text = tweet.full_text

            #Add the 12 variables to the empty list - ith tweet
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                        usercreatedts, tweetcreatedts, retweetcount, text, hashtags]
            
            #Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet
            
            # Increase Counter
            noTweets += 1
        
        #Run Ended
        end_run = time.time()
        duration_run = round((end_run - start_run)/60, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time taken for {} runs to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(920) # 15 minute sleep time
    
    # Once all of the runs have completed, save them to a single csv file
    from datetime import datetime
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    
    # Define path and file name
    filename = "C:\\Users\\mawal\\OneDrive - Binghamton University\\Desktop\\Desktop_Folders\\Teaching\\Python\\" + to_csv_timestamp + '_blm_tweets.csv'
    
    # Store Dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrape is {} minutes'.format(round((program_end - program_start)/60), 2))
        

In [8]:
scrapetweets('Black Lives Matter OR BLM', '2020-11-02', 1000, 2)

no. of tweets scraped for run 1 is 1000
time taken for 1 runs to complete is 1.77 mins
no. of tweets scraped for run 2 is 1000
time taken for 2 runs to complete is 0.96 mins
Scraping has completed!
Total time taken to scrape is 33 minutes
